In [29]:
%%writefile in.nemd
# TODO: Understand this better and figure out the linear fit
# TODO: Make this more intuitive to run
# Sample LAMMPS input script for viscosity of FCC solid
# NEMD via fix deform and fix nvt/sllod

# Settings
variable	    x equal 10
variable	    y equal 10
variable        z equal 10

variable	    a equal 4.05
variable        T equal 2000
variable        P equal 0.0

variable	    srate equal 2.7 # velocity of top edge (Angstroms/ps)

# Problem setup
units		    metal
dimension	    3
boundary        p p p
atom_style	    atomic

lattice         fcc ${a}
region          simbox prism 0 $x 0 $y 0 $z 0.0 0.0 0.0 units lattice
create_box      1 simbox
create_atoms    1 box

pair_style      eam/fs
pair_coeff      * * Cu1.eam.fs Cu
mass            1  63.55

# 1. Equilibration run
velocity        all create $T 97287
fix             1 all npt temp $T $T $(100.0*dt) iso $P $P $(1000.0*dt)

thermo          1000
dump            equilibration all custom 50 dump.equilibration id type x y z vx vy vz 
run	            5000

undump         equilibration
unfix  	        1
reset_timestep  0

# 2. Turn on NEMD shear and equilibrate some more
variable	    xyrate equal ${srate}/ly # Shear rate defined relative to perpendicular dimension (1/ps)

fix		        1 all nvt/sllod temp $T $T $(100.0*dt)

# Perform deformation every 1 timestep
# erate - engineering shear strain rate (1/time units)
fix		        2 all deform 1 xy erate ${xyrate} remap v

compute		    usual all temp
compute		    tilt all temp/deform

thermo          1000
thermo_style	custom step temp c_usual epair etotal press pxy
thermo_modify	temp tilt

dump            nemd_equilibration all custom 50 dump.nemd_equilibration id type x y z vx vy vz
run		        50000
undump          nemd_equilibration
reset_timestep  0

# 3. Data gathering run
# The average in each chunk is computed every 1000 timesteps using 100 samples taken at intervals of 
# 10 timesteps from the preceding timesteps.
# For this case, there seems to be no difference between lower, center, and upper options.
compute         layers_center all chunk/atom bin/1d y center 0.05 units reduced
fix		        3 all ave/chunk 10 100 1000 layers_center vx file profile_center.nemd

# This only takes into account the top and bottom velocity points to fit the linear profile
# It doesn't even use fix 4, which is the only one that actually computes the profile
# The viscosity is calculated from the slope of the linear profile
variable        timestep equal step
variable	    visc equal -pxy/(v_srate/ly)*0.0001 # Computed every timestep. Convert bar.ps to mPa.s

# The average in each chunk is computed every 1000 timesteps using 100 samples taken at intervals of 
# 10 timesteps from the preceding timesteps.
# The running average averages the average values computed every 1000 timesteps.
fix             vprint all print 10 "${timestep} ${visc}" file visc_output.txt screen no
fix		        vave all ave/time 10 100 1000 v_visc ave running start 1000 file visc_run_avg.txt

thermo          1000
thermo_style	custom step temp etotal press pxy ly v_visc f_vave
thermo_modify	temp tilt

dump	        data_gathering all custom 50 dump.data_gathering id type x y z vx vy vz

run		        50000

Overwriting in.nemd


In [18]:
import plotly.graph_objects as go

# Function to vx for a specific timestep using Plotly
def plot_vx(timestep):
    if timestep in chunk_data:
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=chunk_data[timestep]['vx'],
            y=chunk_data[timestep]['coord1'],
            mode='lines+markers',
            name=f'Timestep {timestep}'
        ))
        fig.update_layout(
            title=f'Velocity Profile at Timestep {timestep}',
            yaxis_title='y/l<sub>y</sub>',
            xaxis_title='vx',
            template='plotly_white',
            width=800,
            height=600 
        )
        fig.show()
    else:
        print(f"No data available for timestep {timestep}")

In [19]:
velocity_profile_file = 'profile_center.nemd'

# Initialize dictionaries to store the data for each chunk
chunk_data = {}

# Read the file and parse the data
with open(velocity_profile_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith('#') or line.strip() == '':
            continue
        parts = line.split()
        if len(parts) == 3:
            # This is a timestep line
            current_timestep = int(parts[0])
        elif len(parts) == 4:
            # This is a data line
            chunk = int(parts[0])
            coord = float(parts[1])
            ncount = float(parts[2])
            vx = float(parts[3])
            
            if current_timestep not in chunk_data:
                chunk_data[current_timestep] = {'chunk': [], 'coord1': [], 'ncounts': [], 'vx': []}
                
            chunk_data[current_timestep]['chunk'].append(chunk)
            chunk_data[current_timestep]['coord1'].append(coord)
            chunk_data[current_timestep]['ncounts'].append(ncount)
            chunk_data[current_timestep]['vx'].append(vx)

In [20]:
plot_vx(50000)

## Understand viscosity averaging

In [21]:
import numpy as np

# Read viscosity_output.txt into numpy array ignoring the first line
data = np.loadtxt('visc_output.txt', skiprows=1)

# Reshape the second column to group every 100 values
second_column = data[:, 1][1:]
reshaped = second_column[:].reshape(-1, 100)

# Compute the mean for each group
calculated_averages = reshaped.mean(axis=1)

# Compute the running average
calculated_running_averages = np.cumsum(calculated_averages) / np.arange(1, len(calculated_averages) + 1)

In [23]:
# Read the viscosity_averages.txt file into a numpy array ignoring the first two lines
averages = np.loadtxt('visc_run_avg.txt', skiprows=2)
running_averages = averages[:, 1]

In [24]:
# Compare the LAMMPS calculated running average with the one we computed
np.allclose(running_averages, calculated_running_averages, atol=1e-6)

True

In [25]:
running_averages

array([16079. , 16445.2, 17807.3, 19643. , 21473.3, 22648.4, 23349.3,
       23808.7, 23684.4, 24127.5, 24009.8, 24027.2, 24556.8, 24335.6,
       24227.8, 24033.8, 23303.3, 23384.4, 22751.1, 22989.7, 22714.9,
       22870.1, 23244. , 23351.7, 23459. , 23637.7, 23384.5, 23208.6,
       23021.6, 22725.3, 22310.6, 22236.5, 22256.8, 22124.5, 21696.9,
       21601.3, 21806.8, 21569.2, 21515.8, 21487.8, 21481.1, 21410.8,
       21304. , 21230. , 21232.5, 21120.3, 21076.6, 20987.3, 20895.8,
       20922.5])